Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/classification-text-dnn/auto-ml-classification-text-dnn.png)

# Explain tree-based models on GPU using GPUTreeExplainer


_**This notebook illustrates how to use shap's GPUTreeExplainer on an Azure GPU machine.**_





Problem: Train a tree-based model and explain the model on an Azure GPU machine using the GPUTreeExplainer.

---

## Table of Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Run model explainer locally at training time](#Explain)
    1. Apply feature transformations
    1. Train a binary classification model
    1. Explain the model on raw features
        1. Generate global explanations
        1. Generate local explanations
1. [Visualize explanations](#Visualize)
1. [Deploy model and scoring explainer](#Deploy)
1. [Next steps](#Next)

## Introduction
This notebook demonstrates how to use the GPUTreeExplainer on some simple datasets.  Like the TreeExplainer, the GPUTreeExplainer is specifically designed for tree-based machine learning models, but it is designed to accelerate the computations using NVIDIA GPUs.


Make sure you have executed the [configuration](../../../configuration.ipynb) before running this notebook.

Notebook synopsis:

1. Creating an Experiment in an existing Workspace
2. Configuration and remote run with a GPU machine

## Setup

In [ ]:
import logging
import os
import shutil

import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.run import Run
from azureml.core.model import Model

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [ ]:
print("This notebook was created using version 1.42.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

As part of the setup you have already created a <b>Workspace</b>. To run the script, you also need to create an <b>Experiment</b>. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [ ]:
ws = Workspace.from_config()

# Choose an experiment name.
experiment_name = 'gpu-tree-explainer'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [ ]:
import os
import shutil

project_folder = './azureml-shap-gpu-tree-explainer'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('gpu_tree_explainer.py', project_folder)

## Set up a compute cluster
This section uses a user-provided compute cluster (named "gpu-shap-cluster" in this example). If a cluster with this name does not exist in the user's workspace, the below code will create a new cluster. You can choose the parameters of the cluster as mentioned in the comments.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

num_nodes = 1

# Choose a name for your cluster.
amlcompute_cluster_name = "gpu-shap-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6",
                                                           # To use GPUTreeExplainer, select a GPU such as "STANDARD_NC6" 
                                                           # or similar GPU option
                                                           # available in your workspace
                                                           max_nodes = num_nodes)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### Configure & Run

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# Create a new RunConfig object
run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute target created in previous step
run_config.target = amlcompute_cluster_name

from azureml.core import Environment

environment_name = "shap-gpu-tree"

env = Environment(environment_name)

env.docker.enabled = True
env.docker.base_image = None
env.docker.base_dockerfile = """
FROM rapidsai/rapidsai:cuda10.0-devel-ubuntu18.04
RUN apt-get update && \
apt-get install -y fuse && \
apt-get install -y build-essential && \
apt-get install -y python3-dev && \
source activate rapids && \
apt-get install -y g++ && \
printenv && \
echo "which nvcc: " && \
which nvcc && \
pip install azureml-defaults && \
pip install azureml-telemetry && \
cd /usr/local/src && \
git clone https://github.com/slundberg/shap && \
cd shap && \
mkdir build && \
python setup.py install --user && \
pip uninstall -y xgboost && \
rm /conda/envs/rapids/lib/libxgboost.so && \
pip install xgboost==1.4.2
"""

env.python.user_managed_dependencies = True

from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder, 
                      script='gpu_tree_explainer.py', 
                      compute_target=amlcompute_cluster_name,
                      environment=env) 
run = experiment.submit(config=src)
run